In [62]:
%pip install -U transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [72]:
from transformers import GPT2Tokenizer, GPT2Model
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2", return_dict_in_generate=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
text = "Oh my god why did you do"
encoded_input = tokenizer(text, return_tensors='pt')
output = F.softmax(gpt2(**encoded_input, return_dict=True).logits[:, -1, :])

# We are able to extract next token probs this way.

for token in reversed(torch.argsort(output)[0][-3:]):
    print(tokenizer.decode(token))
    print(output[0][token])

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


 that
tensor(0.5987, grad_fn=<SelectBackward>)
 this
tensor(0.2333, grad_fn=<SelectBackward>)
 it
tensor(0.0974, grad_fn=<SelectBackward>)


In [78]:
def extract_token_probs_from_text(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = F.softmax(gpt2(**encoded_input, return_dict=True).logits[:, -1, :], dim=-1)
    return {tokenizer.decode(i) : output[0][i] for i in range(len(output))}

In [90]:
from time import time

start = time()

for i in range(100):
    if i % 10 == 0:
        print(f"{i + 1}th iteration".ljust(30) + f"{(time() - start):.3f}")
    extract_token_probs_from_text("why did you do")

print()
print(f"Overall time: ".ljust(30) + f"{(time() - start):.3f}")

1th iteration                 0.000
11th iteration                0.670
21th iteration                1.341
31th iteration                2.010
41th iteration                2.672
51th iteration                3.340
61th iteration                4.027
71th iteration                4.691
81th iteration                5.360
91th iteration                6.077

Overall time:                 6.752


In [83]:
# Мысль: во время энкодинга, кажется, можно глядеть сразу на несколько шагов вперёд, и отталкиваться от этой информации, чтобы оперировать батчами и ускорить процесс.
# Во время декодинга, кажется, такое не выйдет :(